# Bonus: Secrets in Remote Runs

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()

In [ ]:
print("Workspace name: " + ws.name, 
      "Azure region: " + ws.location, 
      "Subscription id: " + ws.subscription_id, 
      "Resource group: " + ws.resource_group, sep = "\n")

Sometimes, you may have to pass a secret to a remote run, for example username and password to authenticate against external data source.

Azure ML SDK enables this use case through Key Vault associated with your workspace. The workflow for adding a secret is following.

On local computer:

 1. Read in a local secret, for example from environment variable or user input. To keep them secret, do not insert secret values into code as hard-coded strings.
 2. Obtain a reference to the keyvault
 3. Add the secret name-value pair in the key vault.
 
The secret is then available for remote runs as shown further below.

__Note__: The _azureml.core.keyvault.Keyvault_ is different from _azure.keyvault_ library. It is intended as simplified wrapper for setting, getting and listing user secrets in Workspace Key Vault.

In [ ]:
import os, uuid

# Define a secret locally --> Use random UUID as a substitute for real secret.
local_secret = os.environ.get(
    key="LOCAL_SECRET",
    default = str(uuid.uuid4())
)

In [ ]:
# Get default workspace key vault
keyvault = ws.get_default_keyvault()

# Set secret in key vault
keyvault.set_secret(
    name="secret-name",
    value=local_secret
)

The _set_secret_ method adds a new secret if one doesn't exist, or updates an existing one with new value.
You can list secret names you've added. This method doesn't return the values of the secrets.

In [ ]:
keyvault.list_secrets()

You can retrieve the value of the secret, and validate that it matches the original value. 

__Note__: This method returns the secret value. Take care not to write the the secret value to output.

In [ ]:
retrieved_secret = keyvault.get_secret(name="secret-name")
local_secret == retrieved_secret

In submitted runs on local and remote compute, you can use the get_secret method of Run instance to get the secret value from Key Vault. 

The method gives you a simple shortcut: the Run instance is aware of its Workspace and Keyvault, so it can directly obtain the secret without you having to instantiate the Workspace and Keyvault within remote run.

__Note__: This method returns the secret value. Take care not to write the secret to output.

For example, let's create a simple script _get_secret.py_ that gets the secret we set earlier. In an actual appication, you would use the secret, for example to access a database or other password-protected resource.

Let's first create a folder for the Python script.

In [ ]:
TRAIN_FOLDER_NAME = "train"
TRAIN_FILE_NAME = "train.py"

In [ ]:
import os

os.makedirs(
    name=os.path.join(".", TRAIN_FOLDER_NAME),
    exist_ok=True
)

Now, let's create a training script.

**TASK**: There is some code missing below. Please complete the script.

Hint: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.keyvault(class)?view=azure-ml-py

In [ ]:
%%writefile $TRAIN_FOLDER_NAME/$TRAIN_FILE_NAME

from azureml.core import Run


aml_run = Run.get_context()


def main():
    # Get secret value
    secret_value = aml_run.get_secret(name="secret-name")
    print("Got secret value {} , but don't write it out!".format(len(secret_value) * "*"))


if __name__ == "__main__":
    main()

Then, submit the script as a `Estimator`, and find the obfuscated secret value in run output. You can use the same approach to other kinds of runs, such as Estimator ones.

**TASK**: Fill in the missing values

Hint: https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.estimator.estimator?view=azure-ml-py

In [ ]:
from azureml.train.estimator import Estimator

estimator = Estimator(
    source_directory=TRAIN_FOLDER_NAME,
    entry_script=TRAIN_FILE_NAME,
    compute_target="local"
)

Let's create an experiment to submit the `Estimator`.

**TASK:** Create an experiment, so that we can submit the estimator afterwards.

Hint: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.experiment(class)?view=azure-ml-py

In [ ]:
from azureml.core import Experiment

experiment = Experiment(
    workspace=ws,
    name="bonus-secrets"
)

Now we are ready to submit our experiment with the configuration that we set above. 
You can follow the output here in the notebook or also look at the run in the portal UI as before.

You will note that one of the first steps is to pull the docker base images and build the training container based on it.
As our "training" script does not really do any training here, the setup process takes much longer than the following execution itself.

**Task:** Submit the estimator.

Hint: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.experiment(class)?view=azure-ml-py

In [ ]:
run = experiment.submit(
    config=estimator,
    tags={}
)
run

In [ ]:
run.wait_for_completion(
    show_output=True,
    wait_post_processing=True
)

Furthermore, you can set and get multiple secrets using set_secrets and get_secrets methods.

## 9. Summary

This showed you how you can use secrets in local and remote runs without provoding them as an argument to the Python script.  

## 10. Bonus

In case you still have time left, here are a few more optional things you can try to implement in the notebook above:

- Set multiple and obtain multiple secrets at once.
- Load a set of secrets, wehereas some of them are not available.
- Delete the secrets you defined in the Key Vault.